# Forward Stepwise Selection Example

Here, we compare the results from 2 ways to perform forward stepwise selection. The variable that most decreases the residual sum of squared error (RSS) can be selected in the following ways: 
1. Solving least-squares problems for all potential additional regressors at each step as is commonly done (e.g. in ALAMO)
2. Selecting the variable with greatest abosolute correlaction to the residual (as in the "Least Angle Regression" (Efron, Hastie, Johnston, & Tibshirani, 2004). Available: http://statweb.stanford.edu/~tibs/ftp/lars.pdf  
2b. Using method 2, as implemented in Tibshirani's "best-subset" package. Available: https://github.com/ryantibs/best-subset/blob/master/bestsubset.pdf

In [10]:
from RandomizedRoundingforBSS import * 
import numpy as np 
import pandas as pd 
from copy import copy 

#Generate Data

p = 23 #Number potential regressors
n = 40 #Number of data points 
DM = BuildDataArrays_and_OptimizationModels(n, p, 
            0, 1, 1,
            0.1, 2, 2)
LAU = LinAlgandUpdates(x=DM.x_train, y=DM.y_train)

## 1. Solving Least Squares 

In [11]:
regressors = np.zeros(p) #Null model 
step_sequence = []
objective_sequence = []

while False in (regressors==1): #while not all regressors selected 
    opt_step_obj = 1000000
    for regressor in range(p): #solve least squares problem for each possible regressor 
        step_regressors = copy(regressors)
        if step_regressors[regressor] == 0: 
            step_regressors[regressor] = 1 
            test_step_obj, _, _ = LAU.evaluate_obj(step_regressors, 0, 'arbitrary', n) 
            if test_step_obj < opt_step_obj: 
                opt_step_obj = test_step_obj
                opt_regressors = copy(step_regressors)
                step = copy(regressor)
    regressors = copy(opt_regressors)
    step_sequence.append(step)
    objective_sequence.append(opt_step_obj)
    
print('Sequence of Regressors Added:', step_sequence)
print('Sequence of RSS:', objective_sequence)

Sequence of Regressors Added: [0, 22, 16, 14, 4, 12, 17, 19, 18, 20, 3, 7, 2, 10, 1, 8, 5, 11, 6, 13, 15, 9, 21]
Sequence of RSS: [29.666003918986146, 25.61738552082976, 23.064875272804283, 20.924538648074375, 19.01969433731825, 17.117704964904547, 14.883156915467518, 13.537549623398428, 12.465669897067354, 11.286964383868195, 10.40777034616362, 9.56696289119194, 9.120637871956077, 8.682208765261063, 8.444878837582822, 8.008391937153844, 7.6949112537568825, 7.489188912701578, 7.311282739785581, 7.113290880818886, 6.955656320650854, 6.752283037407808, 6.59582851215833]


## 2. Maximizing Absolute Correlation to Residual

In [12]:
regressors = np.zeros(p) #Null model 
step_sequence_2 = []
objective_sequence_2 = []

model = np.zeros((n,1)) #initial null model 
x = copy(DM.x_train)
xT = copy(x.T) #Transpose of design matrix 
while False in (regressors==1):
    if len(step_sequence_2) != 0: #if model non-empty 
        for row in range(xT.shape[0]): #orthogonalize other predictors w.r.t. currently chosen predictor  
            if row != step_regressor:
                m = (xT[row].dot(xT[step_regressor]))/(sum(i**2 for i in xT[step_regressor]))*(xT[step_regressor])
                xT[row] = xT[row] - m.flatten() 
    correlation = xT.dot(DM.y_train.flatten() - model.flatten()) #calculate correlation vector 
    step_regressor = np.argmax(np.abs(correlation)) #find regressor with greatest abs correlation 
    step_sequence_2.append(step_regressor)
    regressors[step_regressor] = 1 #add regressor 
    test_step_obj, B_ols, _ = LAU.evaluate_obj(regressors, 0, 'arbitrary', n) #calculate new RSS
    objective_sequence_2.append(test_step_obj)
    B = np.zeros(p)
    j = 0 
    for i in range(len(regressors)): 
        if regressors[i] == 1: 
            B[i] = B_ols[j]
            j+=1 
    model = model.flatten() + B[step_regressor]*xT[step_regressor] #update model 
    model = model.reshape((n,1))


print('Sequence of Regressors Added:', step_sequence_2)
print('Sequence of RSS:', objective_sequence_2)


Sequence of Regressors Added: [0, 22, 16, 14, 4, 12, 17, 19, 7, 18, 20, 3, 2, 10, 1, 8, 11, 6, 5, 15, 13, 9, 21]
Sequence of RSS: [29.66600391898614, 25.617385520829778, 23.06487527280429, 20.924538648074364, 19.019694337318235, 17.117704964904544, 14.883156915467508, 13.53754962339842, 12.481457239611832, 11.427064729063517, 10.481458990054497, 9.56696289119193, 9.120637871956065, 8.682208765261054, 8.444878837582813, 8.008391937153835, 7.74360546597418, 7.562451200207468, 7.311282739785576, 7.143548345064097, 6.955656320650851, 6.752283037407811, 6.5958285121583335]


## 2b. Tibshirani 'best-subset' Package Implementation (Using Method 2 Above)

In [33]:
FSsoln = bestsubset_tibshirani.fs(DM.x_train_R, DM.y_train_R,intercept=False)
print('Sequence of Regressors Added:', np.array(FSsoln[0],dtype=int)-1)

Sequence of Regressors Added: [ 0 22 16 14  4 12 17 19 18 20  3  7  2 10  1  8  5 11  6 13 15  9 21]


## Summary
Here, we summarize the order of regressor addition given by the methods shown above. 

In [34]:
Steps = pd.DataFrame({'Method 1 (LSTSQ)': step_sequence, 'Method 2 (Abs. Correlation)': step_sequence_2, 
                      'Method 2b (R Package)': np.array(FSsoln[0],dtype=int)-1}, index=['Step '+ str(i+1) for i in range(p)])
print(Steps)

         Method 1 (LSTSQ)  Method 2 (Abs. Correlation)  Method 2b (R Package)
Step 1                  0                            0                      0
Step 2                 22                           22                     22
Step 3                 16                           16                     16
Step 4                 14                           14                     14
Step 5                  4                            4                      4
Step 6                 12                           12                     12
Step 7                 17                           17                     17
Step 8                 19                           19                     19
Step 9                 18                            7                     18
Step 10                20                           18                     20
Step 11                 3                           20                      3
Step 12                 7                            3          